In [1]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

C:\Users\ASDF\anaconda3\envs\gnn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model Construction 

In [2]:
class GCNConv(MessagePassing):
    # Applies a linear tranformation to the incoming data
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Multiply with weights
        x = self.lin(x)

        # Step 3: Calculate the normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4: Propagate the embeddings to the next layer
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x,
                              norm=norm)

    def message(self, x_j, norm):
        # Normalize node features.
        return norm.view(-1, 1) * x_j


In [3]:
class Net(torch.nn.Module):

    def __init__(self, dataset):
        super(Net, self).__init__()
        # two convolution layers of GCN
        # first represnt the input and the second represnt the output layer
        # dataset.num_node_features== number of input
        # 16 hidden layers
        # dataset.num_classes==outputclass
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
    # represent the weights updates
    # weights in output
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
#         print(x)
#         print(edge_index)
        x = self.conv1(x, edge_index)
#         print('first convo X:',x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
#         print('sec convo X:',x)

        return F.log_softmax(x, dim=1)



In [4]:
# def plot_dataset(dataset):
#     edges_raw = dataset.data.edge_index.numpy()
#     edges = [(x, y) for x, y in zip(edges_raw[0, :], edges_raw[1, :])]
#     labels = dataset.data.y.numpy()

#     G = nx.Graph()
#     G.add_nodes_from(list(range(np.max(edges_raw))))
#     G.add_edges_from(edges)
#     plt.subplot(111)
#     options = {
#                 'node_size': 30,
#                 'width': 0.2,
#     }
#     nx.draw(G, with_labels=False, node_color=labels.tolist(), cmap=plt.cm.tab10, font_weight='bold', **options)
#     plt.show()


In [5]:
def test(data, train=True):
    model.eval()

    correct = 0
    pred = model(data).max(dim=1)[1]

    if train:
        correct += pred[data.train_mask].eq(data.y[data.train_mask]).sum().item()
        return correct / (len(data.y[data.train_mask]))
    else:
        correct += pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
        return correct / (len(data.y[data.test_mask]))

In [6]:
def train(data, plot=False):
    train_accuracies, test_accuracies = list(), list()
    for epoch in range(100):
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            train_acc = test(data)
            test_acc = test(data, train=False)

            train_accuracies.append(train_acc)
            test_accuracies.append(test_acc)
            print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Test Acc: {:.5f}'.
                  format(epoch, loss, train_acc, test_acc))

    # torch.save(model.state_dict(), 'new_model_weights.pth')

In [7]:
# if plot:
#     plt.plot(train_accuracies, label="Train accuracy")
#     plt.plot(test_accuracies, label="Validation accuracy")
#     plt.xlabel("# Epoch")
#     plt.ylabel("Accuracy")
#     plt.legend(loc='upper right')
#     plt.show()

# Dataset loading 

In [8]:
dataset = Planetoid(root='cora_dataset', name='Cora')


In [9]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [10]:
dataset.data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [11]:
dataset.data.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [12]:
dataset.data.y

tensor([3, 4, 4,  ..., 3, 3, 3])

In [13]:
data = dataset[0].to(device)

NameError: name 'device' is not defined

In [14]:
# dataset.info()

In [15]:
print(dataset)
print("number of graph:" ,len(dataset))
print("number of classes:", dataset.num_classes)
print("number of node features:", dataset.num_node_features)
# print("number of edge features:\t",dataset.num_edge_features)

Cora()
number of graph: 1
number of classes: 7
number of node features: 1433


In [16]:
print("edge_index shape:\t", dataset.data.edge_index.shape)
print("edge_index=",dataset.data.edge_index)
print('\n')
print("train_mask shape:\t", dataset.data.train_mask.shape)
print("train_mask=",dataset.data.train_mask)
print('\n')
print("x shape:\t", dataset.data.x.shape)
print("Shape of x=",dataset.data.x)
print('\n')
print("y shape:\t", dataset.data.y.shape)
print('y',dataset.data.y)

edge_index shape:	 torch.Size([2, 10556])
edge_index= tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


train_mask shape:	 torch.Size([2708])
train_mask= tensor([ True,  True,  True,  ..., False, False, False])


x shape:	 torch.Size([2708, 1433])
Shape of x= tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y shape:	 torch.Size([2708])
y tensor([3, 4, 4,  ..., 3, 3, 3])


In [17]:
print("Total data view:",data)

NameError: name 'data' is not defined

In [18]:
# for i in data:
#     print(i)

In [19]:
model = Net(dataset).to(device)

NameError: name 'device' is not defined

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Train model function call 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train(data, plot=False)

# Prediction on direct model learning

In [ ]:
pred=model(dataset[0])
pred

In [ ]:
print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
print('actual y values____:',dataset[0].y.tolist())

# Model saving and loading

In [ ]:
# saving complete model
torch.save(model, 'Updated_GNN_model_weights.pth')

In [ ]:
# load model
model = torch.load('GNN_model_weights.pth')

# Prediction on loaded model

In [ ]:
y_pred=model(dataset[0])
y_pred

In [ ]:
print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
print('actual y values____:',dataset[0].y.tolist())

# Code End here

In [ ]:
model(dataset.data.x[1],dataset.data.edge_index,dataset.data.)

In [ ]:
    dataset = Planetoid(root='/tmp/Cora', name='Cora')

    print(dataset)
    print("number of graph:" ,len(dataset))
    print("number of classes:", dataset.num_classes)
    print("number of node features:", dataset.num_node_features)
    # print("number of edges:",dataset.num_edge_features_features)

    print("edge_index:\t\t", dataset.data.edge_index.shape)
    print(dataset.data.edge_index)
    print("\n")
    print("train_mask:\t\t", dataset.data.train_mask.shape)
    print(dataset.data.train_mask)
    print("\n")
    print("x:\t\t", dataset.data.x.shape)
    print(dataset.data.x)
    print("\n")
    print("y:\t\t", dataset.data.y.shape)
    print(dataset.data.y)

    # print('x values',dataset[0].x)
    sg_x=dataset[0].x.tolist()[1]
    # print("signal record",dataset[0].x.tolist()[1])
    sg_ed = dataset[0].edge_index.tolist()[1]
    # print(sg_ed )
    # print('y values',dataset[0].y)
    # print('complete values',dataset[0])

    # plot_dataset(dataset)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(dataset).to(device)
    data = dataset[0].to(device)
    print("data check",data)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # train(data, plot=True)

    # saving and load with other approch
    # torch.save(model.state_dict(), 'dic_state_save_model.pt')
    # the_model = model()

    # the_model.load_state_dict(torch.load(PATH))
    # ___________
    # saving complete model
    torch.save(model, 'GNN_model_weights.pth')
    # load model
    model = torch.load('GNN_model_weights.pth')
    # print(model.eval())

    # prediction of all datset
    print("test=",dataset[0])
    print("complete dataset: =", dataset)
    
    y_pred=model(dataset[0])
    # print(y_pred)
    print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
    print('actual y values____:',dataset[0].y.tolist())
    print('data_x',data.x)
    print('data_edges',data.edge_index)
    # test_data(x=data.x,edge_index=data.edge_index)
    print('zero......',data.x[0])
    print('last....',data.x[2707])
    # res=model(data.x[])
    # print(res)



    di={
        'predicted':[np.argmax(i) for i in y_pred.tolist()],
        'actual':dataset[0].y,
    }

    #
    # print(model(di))

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

print(dataset)
print("number of graph:" ,len(dataset))
print("number of classes:", dataset.num_classes)
print("number of node features:", dataset.num_node_features)
# print("number of edges:",dataset.num_edge_features_features)

#     print("edge_index:\t\t", dataset.data.edge_index.shape)
#     print(dataset.data.edge_index)
#     print("\n")
#     print("train_mask:\t\t", dataset.data.train_mask.shape)
#     print(dataset.data.train_mask)
#     print("\n")
#     print("x:\t\t", dataset.data.x.shape)
#     print(dataset.data.x)
#     print("\n")
#     print("y:\t\t", dataset.data.y.shape)
#     print(dataset.data.y)

    # print('x values',dataset[0].x)
sg_x=dataset[0].x.tolist()[1]
    # print("signal record",dataset[0].x.tolist()[1])
sg_ed = dataset[0].edge_index.tolist()[1]
    # print(sg_ed )
    # print('y values',dataset[0].y)
    # print('complete values',dataset[0])

    # plot_dataset(dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset).to(device)
data = dataset[0].to(device)
print("data check",data)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # train(data, plot=True)

    # saving and load with other approch
    # torch.save(model.state_dict(), 'dic_state_save_model.pt')
    # the_model = model()

    # the_model.load_state_dict(torch.load(PATH))
    # ___________
    # saving complete model
torch.save(model, 'GNN_model_weights.pth')
    # load model
model = torch.load('GNN_model_weights.pth')
    # print(model.eval())

    # prediction of all datset
print("test=",dataset[0])
print("complete dataset: =", dataset)
    
y_pred=model(dataset[0])
    # print(y_pred)
# print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
# print('actual y values____:',dataset[0].y.tolist())
# print('data_x',data.x)
#     print('data_edges',data.edge_index)
#     # test_data(x=data.x,edge_index=data.edge_index)
#     print('zero......',data.x[0])
#     print('last....',data.x[2707])
#     # res=model(data.x[])
    # print(res)



#     di={
#         'predicted':[np.argmax(i) for i in y_pred.tolist()],
#         'actual':dataset[0].y,
#     }

    #
    # print(model(di))

In [ ]:
# data.x.reshape(1,1433)

In [ ]:
# y_pred=model(data.x)
# y_pred

In [ ]:
print('zero......:',data.x[0])
# print('last......',data.x[2700])
print('y.........:',data.y[0])
print('mask_train zero:',data.train_mask[0])
print('val_mask zero:',data.val_mask[0])
print('test_mask zero:',data.test_mask[0])
print('total_dataset',dataset[0])

In [ ]:
# data.x[0].tolist()

In [ ]:
data.edge_index

In [ ]:
# y_pred=model(data.x[0],data.edge_index,data.y[0])
# y_pred=model(data.x[0],data.edge_index,data.val_mask[0],data.test_mask[0])

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

    print(dataset)
    print("number of graph:" ,len(dataset))
    print("number of classes:", dataset.num_classes)
    print("number of node features:", dataset.num_node_features)
    # print("number of edges:",dataset.num_edge_features_features)

    print("edge_index:\t\t", dataset.data.edge_index.shape)
    print(dataset.data.edge_index)
    print("\n")
    print("train_mask:\t\t", dataset.data.train_mask.shape)
    print(dataset.data.train_mask)
    print("\n")
    print("x:\t\t", dataset.data.x.shape)
    print(dataset.data.x)
    print("\n")
    print("y:\t\t", dataset.data.y.shape)
    print(dataset.data.y)

    # print('x values',dataset[0].x)
    sg_x=dataset[0].x.tolist()[1]
    # print("signal record",dataset[0].x.tolist()[1])
    sg_ed = dataset[0].edge_index.tolist()[1]
    # print(sg_ed )
    # print('y values',dataset[0].y)
    # print('complete values',dataset[0])

    # plot_dataset(dataset)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(dataset).to(device)
    data = dataset[0].to(device)
    print("data check",data)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # train(data, plot=True)

    # saving and load with other approch
    # torch.save(model.state_dict(), 'dic_state_save_model.pt')
    # the_model = model()

    # the_model.load_state_dict(torch.load(PATH))
    # ___________
    # saving complete model
    torch.save(model, 'GNN_model_weights.pth')
    # load model
    model = torch.load('GNN_model_weights.pth')
    # print(model.eval())

    # prediction of all datset
    print("test=",dataset[0])
    print("complete dataset: =", dataset)
    
    y_pred=model(dataset[0])
    # print(y_pred)
    print('prediction_by_model:',[np.argmax(i) for i in y_pred.tolist()])
    print('actual y values____:',dataset[0].y.tolist())
    print('data_x',data.x)
    print('data_edges',data.edge_index)
    # test_data(x=data.x,edge_index=data.edge_index)
    print('zero......',data.x[0])
    print('last....',data.x[2707])
    # res=model(data.x[])
    # print(res)



    di={
        'predicted':[np.argmax(i) for i in y_pred.tolist()],
        'actual':dataset[0].y,
    }

    #
    # print(model(di))